In [330]:
import pandas as pd

## Institutes

In [331]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df.head()

,id,nome,tipo,posti_regolamentari,posti_non_disponibili,posti_occupati,posti_aggiornati_al,asl,nome_responsabile_asl,cognome_responsabile_asl,...,cognome_direttore,ruolo_direttore,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,personale_polizia_aggiornato_a,personale_amministrativo_aggiornato_al
0,MII179988,Reggio Calabria Arghillà,Casa circondariale,294,0,319,2024-10-05,5 Reggio Calabria,Luciano,Lucania,...,Stendardo,Titolare,131,164,12,32,5,8,2024-07-31,NaN
1,MII172610,Brescia Verziano,Casa di reclusione,71,0,119,2024-10-05,Spedali Civili di Brescia,Luigi,Leone,...,NaN,NaN,79,95,2,0,1,0,2024-07-31,NaN
2,MII172320,Busto Arsizio,Casa circondariale,240,14,431,2024-10-05,Busto Arsizio,Ezia,Iorio,...,Pitaniello,Titolare,193,190,16,21,5,5,2024-07-31,NaN
3,MII173712,Como,Casa circondariale,226,1,426,2024-10-05,Como,Giuseppe,Carrano,...,Rinaldi,Titolare,208,216,18,23,6,6,2024-07-31,NaN
4,MII173747,Cremona,Casa circondariale,394,10,586,2024-10-05,CREMONA,Rossano,Botto,...,Padula,Titolare,188,202,15,23,5,6,2024-07-31,NaN


## Most recent data

In [332]:
# Get the most recent update
df['posti_aggiornati_al'] = pd.to_datetime(df['posti_aggiornati_al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id')['posti_aggiornati_al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti_reali'] = df_most_recent['posti_regolamentari'] - df_most_recent['posti_non_disponibili']
df_most_recent['tasso_sovraffollamento'] = round(((df_most_recent['posti_occupati'] / df_most_recent['posti_reali'])*100),0)

In [333]:
df_most_recent.value_counts(['posti_aggiornati_al'])

posti_aggiornati_al
2024-10-18             190
dtype: int64

In [334]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id'})

merged_df = pd.merge(df_most_recent, df_info, on='id')

# Adding additional columns
merged_df['posti_reali'] = merged_df['posti_regolamentari'] - merged_df['posti_non_disponibili']

merged_df['tasso_affollamento'] = round((merged_df['posti_occupati'] / merged_df['posti_reali']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id',
    'nome',
    'tasso_affollamento',
    'indirizzo',
    'tipo',
   'posti_regolamentari',
    'posti_non_disponibili',
    'posti_reali',
    'posti_occupati',
    'posti_aggiornati_al',
    'personale_polizia_previsti',
    'personale_polizia_effettivi',
    'personale_polizia_aggiornato_a',
    'personale_amministrativi_previsti',
    'personale_amministrativi_effettivi',
    'personale_amministrativo_aggiornato_al',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head()

,id,nome,tasso_affollamento,indirizzo,tipo,posti_regolamentari,posti_non_disponibili,posti_reali,posti_occupati,posti_aggiornati_al,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,personale_amministrativi_previsti,personale_amministrativi_effettivi,personale_amministrativo_aggiornato_al,latitudine,longitude
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,675,2024-10-18,336,336,2024-09-30,36,29,NaN,44.417913,8.951100
1,MII157783,Alba,82.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,40,2024-10-18,107,88,2024-09-30,14,10,NaN,44.691542,8.025636
2,MII158895,Milano San Vittore,224.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,241,461,1034,2024-10-18,655,562,2024-09-30,45,37,NaN,45.461598,9.166399
3,MII158901,Belluno,130.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,107,2024-10-18,85,80,2024-09-30,14,8,NaN,46.139738,12.219297
4,MII158910,Agrigento,142.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,9,274,390,2024-10-18,197,219,2024-09-30,23,21,NaN,37.319422,13.617396


In [335]:
df_filtered.head()

,id,nome,tasso_affollamento,indirizzo,tipo,posti_regolamentari,posti_non_disponibili,posti_reali,posti_occupati,posti_aggiornati_al,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,personale_amministrativi_previsti,personale_amministrativi_effettivi,personale_amministrativo_aggiornato_al,latitudine,longitude
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,675,2024-10-18,336,336,2024-09-30,36,29,NaN,44.417913,8.951100
1,MII157783,Alba,82.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,40,2024-10-18,107,88,2024-09-30,14,10,NaN,44.691542,8.025636
2,MII158895,Milano San Vittore,224.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,241,461,1034,2024-10-18,655,562,2024-09-30,45,37,NaN,45.461598,9.166399
3,MII158901,Belluno,130.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,107,2024-10-18,85,80,2024-09-30,14,8,NaN,46.139738,12.219297
4,MII158910,Agrigento,142.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,9,274,390,2024-10-18,197,219,2024-09-30,23,21,NaN,37.319422,13.617396


In [336]:
df_filtered['scheda_istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id'] + '">Vai alla scheda istituto.</a>'


df_filtered.head()

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_76363/1106251922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda_istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id'] + '">Vai alla scheda istituto.</a>'


,id,nome,tasso_affollamento,indirizzo,tipo,posti_regolamentari,posti_non_disponibili,posti_reali,posti_occupati,posti_aggiornati_al,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,personale_amministrativi_previsti,personale_amministrativi_effettivi,personale_amministrativo_aggiornato_al,latitudine,longitude,scheda_istituto
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,675,2024-10-18,336,336,2024-09-30,36,29,NaN,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,82.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,40,2024-10-18,107,88,2024-09-30,14,10,NaN,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,224.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,241,461,1034,2024-10-18,655,562,2024-09-30,45,37,NaN,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,130.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,107,2024-10-18,85,80,2024-09-30,14,8,NaN,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,142.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,9,274,390,2024-10-18,197,219,2024-09-30,23,21,NaN,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [337]:
# df_filtered['personale_polizia_mancanti'] = df_filtered['personale_polizia_previsti'] - df_filtered['personale_polizia_effettivi']
# df_filtered['personale_polizia_mancanti_perc'] = round(df_filtered['personale_polizia_mancanti'] / df_filtered['personale_polizia_previsti'] * 100, 2)

# df_filtered.sort_values('personale_polizia_mancanti_perc', ascending=False)

In [338]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [339]:
grouped_df = df.groupby('posti_aggiornati_al').sum(numeric_only=True).reset_index()
grouped_df

,posti_aggiornati_al,posti_regolamentari,posti_non_disponibili,posti_occupati,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,personale_amministrativo_aggiornato_al
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0
5,2024-10-11,51199,4441,61846,31091,34035,3414,4079,978,1001,0.0
6,2024-10-12,51199,4439,61885,31091,34035,3414,4079,978,1001,0.0
7,2024-10-13,51199,4439,61910,31091,34035,3414,4079,978,1001,0.0
8,2024-10-14,51199,4445,61844,31091,34035,3414,4079,978,1001,0.0
9,2024-10-17,50858,4458,61572,30890,33804,3402,4058,974,996,0.0


In [340]:
grouped_df['posti_reali'] = (grouped_df['posti_regolamentari'] - grouped_df['posti_non_disponibili']).round(0)
grouped_df['tasso_sovraffollamento'] = ((grouped_df['posti_occupati'] / grouped_df['posti_reali']))*100

grouped_df.head()


,posti_aggiornati_al,posti_regolamentari,posti_non_disponibili,posti_occupati,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,personale_amministrativo_aggiornato_al,posti_reali,tasso_sovraffollamento
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,46812,132.188328
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,46812,132.256686
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,46755,132.276762
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,46756,132.310292
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,46758,132.261859


In [341]:
grouped_df = grouped_df[['posti_aggiornati_al', 'posti_regolamentari', 'posti_non_disponibili', 'posti_occupati']]
grouped_df['posti_disponibili'] = grouped_df['posti_regolamentari'] - grouped_df['posti_non_disponibili']
grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_76363/1429953430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_df['posti_disponibili'] = grouped_df['posti_regolamentari'] - grouped_df['posti_non_disponibili']
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_76363/1429953430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)


,posti_aggiornati_al,posti_regolamentari,posti_non_disponibili,posti_occupati,posti_disponibili,tasso_affollamento
0,2024-10-05,51196,4384,61880,46812,132.1883
1,2024-10-06,51196,4384,61912,46812,132.2567
2,2024-10-07,51196,4441,61846,46755,132.2768
3,2024-10-09,51195,4439,61863,46756,132.3103
4,2024-10-10,51195,4437,61843,46758,132.2619


In [342]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [343]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome', 'posti_occupati', 'tasso_affollamento', 'personale_polizia_previsti', 'personale_polizia_effettivi', 'personale_polizia_aggiornato_a', 'posti_aggiornati_al']]

df_polizia['personale_polizia_mancanti'] = df_polizia['personale_polizia_previsti'] - df_polizia['personale_polizia_effettivi']

df_polizia.head(2)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_76363/3422003152.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['personale_polizia_mancanti'] = df_polizia['personale_polizia_previsti'] - df_polizia['personale_polizia_effettivi']


,nome,posti_occupati,tasso_affollamento,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,posti_aggiornati_al,personale_polizia_mancanti
0,Genova Marassi,675,126.0,336,336,2024-09-30,2024-10-18,0
1,Alba,40,82.0,107,88,2024-09-30,2024-10-18,19


In [344]:
df_polizia['personale_polizia_mancanti_perceent'] = round(df_polizia['personale_polizia_mancanti'] / df_polizia['personale_polizia_previsti']*100,2)
df_polizia.head(2)


/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_76363/1198721823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['personale_polizia_mancanti_perceent'] = round(df_polizia['personale_polizia_mancanti'] / df_polizia['personale_polizia_previsti']*100,2)


,nome,posti_occupati,tasso_affollamento,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,posti_aggiornati_al,personale_polizia_mancanti,personale_polizia_mancanti_perceent
0,Genova Marassi,675,126.0,336,336,2024-09-30,2024-10-18,0,0.00
1,Alba,40,82.0,107,88,2024-09-30,2024-10-18,19,17.76


In [345]:
df_polizia_clean = df_polizia[df_polizia['posti_occupati'] != 0]
df_polizia_clean.sort_values('personale_polizia_mancanti_perceent', ascending=False).head(2)

,nome,posti_occupati,tasso_affollamento,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,posti_aggiornati_al,personale_polizia_mancanti,personale_polizia_mancanti_perceent
136,Roma Rebibbia IIIª casa,84,64.0,46,31,2024-09-30,2024-10-18,15,32.61
158,Tempio Pausania,172,101.0,146,104,2024-09-30,2024-10-18,42,28.77


In [346]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso_affollamento'] > 120) & (df_polizia_clean['personale_polizia_mancanti_perceent'] > 20)].reset_index(drop=True)
critical_prisons

,nome,posti_occupati,tasso_affollamento,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,posti_aggiornati_al,personale_polizia_mancanti,personale_polizia_mancanti_perceent
0,Ancona Montacuto,328,128.0,175,125,2024-09-30,2024-10-18,50,28.57
1,Augusta,549,161.0,224,175,2024-09-30,2024-10-18,49,21.88
2,Rieti,505,186.0,175,137,2024-09-30,2024-10-18,38,21.71
3,Bolzano,106,125.0,77,61,2024-09-30,2024-10-18,16,20.78
4,Cassino,216,170.0,142,107,2024-09-30,2024-10-18,35,24.65
5,Lucca,76,185.0,91,71,2024-09-30,2024-10-18,20,21.98
6,Matera,188,149.0,125,97,2024-09-30,2024-10-18,28,22.40
7,Pescara,455,167.0,166,122,2024-09-30,2024-10-18,44,26.51
8,Roma Regina Coeli,1096,192.0,480,347,2024-09-30,2024-10-18,133,27.71
9,Rossano,321,122.0,139,109,2024-09-30,2024-10-18,30,21.58


In [347]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [348]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [349]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'posti_aggiornati_al': 'Date', 'tasso_affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [350]:
combined_df = combined_df[['Date', 'tasso_affollamento_ufficiale', 'tasso_affollamento_reale']]

filtered_df = combined_df[combined_df['Date'] > '2024-02-01']


In [351]:
filtered_df.head(2)

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
61,2024-02-29,119.0224,NaN
62,2024-03-31,119.2876,NaN


In [352]:
combined_df = combined_df[['Date', 'tasso_affollamento_ufficiale', 'tasso_affollamento_reale']]
filtered_df = combined_df[combined_df['Date'] > '2024-04-01']
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)

filtered_df.head(2)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
63,2024-04-30,119.7979,NaN
64,2024-05-31,120.1128,NaN


In [353]:
# # Integrating older data
# df3 = pd.read_csv('../old/istituti_data.csv')
# grouped_df3 = df3.groupby('posti_aggiornati_al').sum(numeric_only=True).reset_index()
# grouped_df3 = grouped_df3[['posti_aggiornati_al','posti_regolamentari', 'posti_non_disponibili', 'posti_occupati']]
# grouped_df3['posti_aggiornati_al'] = pd.to_datetime(grouped_df3['posti_aggiornati_al'], errors='coerce')
# grouped_df3['posti_aggiornati_al'] = grouped_df3['posti_aggiornati_al'].dt.strftime('%Y-%m-%d')
# grouped_df3['posti_disponibili'] = grouped_df3['posti_regolamentari'] - grouped_df3['posti_non_disponibili']
# grouped_df3['tasso_affollamento_reale'] = round((grouped_df3['posti_occupati'] / grouped_df3['posti_disponibili']) * 100, 4)
# grouped_df3 = grouped_df3[['posti_aggiornati_al', 'tasso_affollamento_reale']]
# grouped_df3.rename(columns={'posti_aggiornati_al': 'Date'}, inplace=True)
# grouped_df3 = grouped_df3.drop_duplicates()
# grouped_df3 = grouped_df3[grouped_df3['Date'] != '2024-10-05'] # Already present in current data

# combined_df2 = pd.merge(filtered_df, grouped_df3, on='Date', how='outer')
# combined_df2['tasso_affollamento_reale_x'] = combined_df2['tasso_affollamento_reale_x'].fillna(combined_df2['tasso_affollamento_reale_y'])
# combined_df2 = combined_df2.drop(columns=['tasso_affollamento_reale_y'])

# combined_df2 = combined_df2.sort_values('Date')
# filtered_df2 = combined_df2[combined_df2['Date'] > '2024-01-01']
# filtered_df2.rename(columns={'tasso_affollamento_reale_x': 'tasso_affollamento_reale'}, inplace=True)

# filtered_df2.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)



In [354]:
# Apply linear interpolation for missing values
filtered_df2['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df2['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df2['tasso_affollamento_reale (interpolated)'] = round((filtered_df2['tasso_affollamento_reale'].interpolate(method='linear')),4)
filtered_df2 = filtered_df2[filtered_df2['Date'] > '2024-06-01']

filtered_df2


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale,tasso_affollamento_ufficiale (interpolated),tasso_affollamento_reale (interpolated)
21,2024-06-17,NaN,130.4216,NaN,130.4216
5,2024-06-30,119.9984,NaN,119.9984,130.4643
6,2024-07-31,119.3841,NaN,119.3841,130.5069
23,2024-08-03,NaN,130.5496,120.0247,130.5496
22,2024-08-13,NaN,131.1777,120.6652,131.1777
7,2024-08-31,121.3058,NaN,121.3058,131.4767
8,2024-09-30,120.8337,NaN,120.8337,131.7756
19,2024-10-01,NaN,132.0746,120.8337,132.0746
9,2024-10-05,NaN,132.1883,120.8337,132.1883
10,2024-10-06,NaN,132.2567,120.8337,132.2567


In [355]:
# filtered_df2.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)
